In [1]:
import os

---

Exploration of *4932.actions.v4.0.tsv*:

**2015 December 2**

According to the README, the scores are "multiplied by 1000." From Kuhn, *et al.* (2014), high-confidence scores have a cutoff of 0.7. 

All entries in the "mode" column are of the following:
* activation
* binding
* catalysis
* inhibition
* reaction

Entries in the "action" column are either "activation" or "inhibition."

If "mode" entry is "activation" or "inhibition", then the corresponding entry in the "action" column is also "activation" or "inhibition", respectively. 

If "action" entry is "activation" or "inhibition", then the corresponding entry in the "mode" column (other than "activation" or "inhibition") is "binding."